In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import PIL
from random import shuffle
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
import pickle

In [3]:
with open('dataconv.pt', 'rb') as f:
    data = pickle.load(f)
    
with open('labels.pt', 'rb') as f:
    labels = pickle.load(f)

In [4]:
class VGG16(nn.Module):
    def __init__(self,dropout=0.5):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3,padding=1)
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(7168,4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 4)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv1(x)))
        x = self.dropout(torch.relu(self.conv2(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv3(x)))
        x = self.dropout(torch.relu(self.conv4(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv5(x)))
        x = self.dropout(torch.relu(self.conv6(x)))
        x = self.dropout(torch.relu(self.conv6(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv7(x)))
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.mp(x)
        x = self.dropout(torch.relu(self.conv8(x)))
        x = self.dropout(torch.relu(self.conv9(x)))
        x = self.dropout(torch.relu(self.conv9(x)))
        x = self.mp(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return torch.log_softmax(x,1)


In [ ]:
class AutoUXDataset(Dataset):
    def __init__(self, data, labels,isTrain=True):
        self.data = data
        self.labels = labels
        self.isTrain = isTrain
        self.setTest(0)

    def __len__(self):
        if self.isTrain:
            return len(self.trainX)
        else:
            return len(self.testX)

    def setTest(self,testI):
        d = []
        l = []
        for i in range(5):
            if i != testI:
                d = d + data[i];
                l = l + labels[i]
            else:
                self.testX = torch.from_numpy(np.asarray(data[i],dtype=np.uint8)).type('torch.FloatTensor')
                self.testY = torch.from_numpy(np.asarray(labels[i],dtype=np.uint8)).type('torch.LongTensor')
                
        self.trainX = torch.from_numpy(np.asarray(d,dtype=np.uint8)).type('torch.FloatTensor')
        self.trainY = torch.from_numpy(np.asarray(l,dtype=np.uint8)).type('torch.LongTensor')
    
    def __getitem__(self, idx):
        if self.isTrain:
            return {"img" : self.trainX[idx], "label": self.trainY[idx]}
        else:
            return {"img" : self.testX[idx], "label": self.testY[idx]}

In [ ]:
def train_batch(model,crit,opt,x, y):
    opt.zero_grad()
    y_pred = model(x)
    loss = crit(y_pred, y)
    loss.backward()
    opt.step() 
    return loss.item(),torch.sum(torch.max(y_pred, 1)[1] == y).item() / len(y)

In [ ]:
def train(model,crit,opt,dataload,disable=False):
    # Training loop
    dloss, dacc = 0,0
    for s in tqdm(dataload,disable=disable):
        x, y = s.values()
        x = x.cuda()
        y = y.cuda()
        lo, acc = train_batch(model,crit,opt,x,y)
        dloss += lo
        dacc += acc
    dloss /= len(dataload)
    dacc /= len(dataload)
    return dloss,dacc

In [ ]:
def test_batch(model,crit,x, y):
    y_pred = model(x)
    loss = crit(y_pred, y)
    return loss.item(),torch.sum(torch.max(y_pred, 1)[1] == y).item() / len(y)

In [ ]:
def test(model,crit,dataload,disable=False):
    # Training loop
    dloss, dacc = 0,0
    with torch.no_grad():
        for s in tqdm(dataload,disable=disable):
            x, y = s.values()
            x = x.cuda()
            y = y.cuda()
            lo, acc = test_batch(model,crit,x,y)
            dloss += lo
            dacc += acc
    dloss /= len(dataload)
    dacc /= len(dataload)
    return dloss,dacc

In [ ]:
avedloss, avedacc, avevloss, avevacc = 0,0,0,0
my_data = AutoUXDataset(data,labels,True)
my_test = AutoUXDataset(data,labels,False)

batch_size = 32
disable = False
epochs = 50

for i in range(5):
    model = VGG16(dropout=0.5).cuda()
    optimizer = optim.Adam(model.parameters(), lr=3e-4,weight_decay=0.1)
    #amp_handle.wrap_optimizer(optimizer)
    criterion = nn.CrossEntropyLoss()
    #amp_handle = amp.init()
    dlosscurve = []
    dacccurve = []
    vlosscurve = []
    vacccurve = []
    print("Fold %d" % (i + 1))
    my_data.setTest(i)
    my_test.setTest(i)
    
    my_loader = DataLoader(my_data, batch_size=batch_size,
                            shuffle=True)
    my_test_loader = DataLoader(my_test, batch_size=batch_size,
                            shuffle=True)
    for j in range(epochs):
        dloss, dacc = train(model,criterion,optimizer,my_loader,disable)
        vloss, vacc = test(model,criterion,my_test_loader,disable)
        dlosscurve.append(dloss)
        dacccurve.append(dacc)
        vlosscurve.append(vloss)
        vacccurve.append(vacc)
        if j % 10 == 0:
            print("Epoch Train Loss: {:.6f}  Epoch Train Accuracy: {:.6f}  Epoch Test Loss: {:.6f}  Epoch Test Accuracy: {:.6f}".format(dloss,dacc * 100,vloss,vacc * 100))
    df = pd.DataFrame(data={"train_loss": dlosscurve, "val_loss": vlosscurve})
    df.plot.line()
    avedloss += dloss
    avedacc += dacc
    avevloss += vloss
    avevacc += vacc
avedloss /= 5
avedacc /= 5
avevloss /= 5
avevacc /= 5
print("Average Train Loss: {:.6f}\nAverage Train Accuracy: {:.6f}\nAverage Test Loss: {:.6f}\nAverage Test Accuracy: {:.6f}\n".format(avedloss,avedacc,avevloss,avevacc))

Fold 1


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.64it/s]


Epoch Train Loss: 1.066489  Epoch Train Accuracy: 49.020011  Epoch Test Loss: 0.713654  Epoch Test Accuracy: 49.320652


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.80it/s]


Epoch Train Loss: 0.703562  Epoch Train Accuracy: 50.849781  Epoch Test Loss: 0.697720  Epoch Test Accuracy: 52.875906


...

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.70it/s]


Fold 3


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.48it/s]


Epoch Train Loss: 1.090993  Epoch Train Accuracy: 45.182292  Epoch Test Loss: 0.790704  Epoch Test Accuracy: 47.881944


 38%|███████████████████████████████▏                                                   | 9/24 [00:04<00:07,  2.06it/s]